#### I am crating this notebook since I realised that I should consolidate all the preprocessing of the data before training

In [1]:
import pandas as pd
import numpy as np
import os
import re
from ast import literal_eval

In [2]:
df = pd.read_csv('/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/data/cards_data.csv')
df.sample(5)

,id,name,cmc,mana_cost,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,textless,rarity,reserved,released_at,booster,usd
6179,082690e8-07b9-4a91-b779-e0123a69ee12,Drownyard Behemoth,9.0,{9},['C'],False,['Creature'],"['Eldrazi', 'Crab']",Flash (You may cast this spell any time you co...,"['Emerge', 'Flash']",5,7,False,uncommon,False,2016-07-22,True,0.07
11036,6a4e4be5-e057-4b50-9f86-76bc0b9987de,Infernal Tutor,2.0,{1}{B},['B'],False,['Sorcery'],['none'],Reveal a card from your hand. Search your libr...,['Hellbent'],non-creature,non-creature,False,rare,False,2006-05-05,True,6.17
10906,e71a6bd3-7478-4c3a-8ae0-98352ce28492,Imperial Seal,1.0,{B},['B'],False,['Sorcery'],['none'],"Search your library for a card, then shuffle a...",['no keywords'],non-creature,non-creature,False,mythic,False,2022-07-08,True,72.58
12737,4ecb1362-9a67-4d4c-8d69-9ac2ebf4d0b0,Lifeblood,4.0,{2}{W}{W},['W'],False,['Enchantment'],['none'],Whenever a Mountain an opponent controls becom...,['no keywords'],non-creature,non-creature,False,rare,True,1994-06-01,True,33.40
18420,2cda35eb-ae42-43be-9015-3c468c7ebede,Rotwidow Pack,4.0,{2}{B}{G},"['B', 'G']",False,['Creature'],['Spider'],"Reach\n{3}{B}{G}, Exile a creature card from y...",['Reach'],2,4,False,uncommon,False,2019-06-14,True,0.28


In [3]:
df.drop(columns=['id', 'mana_cost', 'textless'], inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26052 entries, 0 to 26051
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         26052 non-null  object 
 1   cmc          26052 non-null  float64
 2   colors       26052 non-null  object 
 3   legendary    26052 non-null  bool   
 4   types        26052 non-null  object 
 5   sub_types    26052 non-null  object 
 6   oracle_text  25099 non-null  object 
 7   keywords     26052 non-null  object 
 8   power        26052 non-null  object 
 9   toughness    26052 non-null  object 
 10  rarity       26052 non-null  object 
 11  reserved     26052 non-null  bool   
 12  released_at  26052 non-null  object 
 13  booster      26052 non-null  bool   
 14  usd          25996 non-null  float64
dtypes: bool(3), float64(2), object(10)
memory usage: 2.5+ MB


In [5]:
df['oracle_text'].fillna('notext', inplace=True)

In [6]:
df['sub_types'].fillna("['none']", inplace=True)

In [7]:
df['types'] = df['types'].apply(literal_eval)
df['sub_types'] = df['sub_types'].apply(literal_eval)
df['keywords'] = df['keywords'].apply(literal_eval)

In [8]:
df.drop(df[df['colors']=='[nan]'].index, inplace=True)
df = df.reset_index(drop=True)
df.head()

,name,cmc,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,rarity,reserved,released_at,booster,usd
0,+2 Mace,2.0,['W'],False,[Artifact],[Equipment],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,[Equip],non-creature,non-creature,common,False,2021-07-23,True,0.02
1,Aarakocra Sneak,4.0,['U'],False,[Creature],"[Bird, Rogue]",Flying\nWhen Aarakocra Sneak enters the battle...,[Flying],1,4,common,False,2022-06-10,True,0.06
2,Abaddon the Despoiler,5.0,"['B', 'R', 'U']",True,[Creature],"[Astartes, Warrior]",Trample\nMark of Chaos Ascendant — During your...,"[Mark of Chaos Ascendant, Trample]",5,5,mythic,False,2022-10-07,False,2.81
3,Abandoned Outpost,0.0,['No Colors'],False,[Land],[none],Abandoned Outpost enters the battlefield tappe...,[no keywords],non-creature,non-creature,common,False,2001-10-01,True,0.14
4,Abandoned Sarcophagus,3.0,['C'],False,[Artifact],[none],You may cast spells that have a cycling abilit...,[no keywords],non-creature,non-creature,rare,False,2020-04-17,False,0.09


In [9]:
df['colors'].replace("['No Colors']", "['N']", inplace=True)
df['colors']=df['colors'].apply(literal_eval)

In [10]:
df['power'] = df['power'].replace('non-creature', 0)
df['toughness'] = df['toughness'].replace('non-creature', 0)

In [11]:
df.drop(df[df['types'].apply(lambda x: '//' in x)].index, inplace=True)
df = df.reset_index(drop=True)

In [12]:
df['power'] = pd.to_numeric(df['power'], errors='coerce')
df['toughness'] = pd.to_numeric(df['toughness'], errors='coerce')
df['power'] = df['power'].fillna(-1).astype(int)
df['toughness'] = df['toughness'].fillna(-1).astype(int)

In [13]:
df['released_at'] = pd.to_datetime(df['released_at'])
reference_date = df['released_at'].min()
df['released_at'] = (df['released_at'] - reference_date).dt.days

In [14]:
df['cmc'] = df['cmc'].astype(int)

In [15]:
df.drop(columns='name', inplace=True)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25545 entries, 0 to 25544
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   cmc          25545 non-null  int64  
 1   colors       25545 non-null  object 
 2   legendary    25545 non-null  bool   
 3   types        25545 non-null  object 
 4   sub_types    25545 non-null  object 
 5   oracle_text  25545 non-null  object 
 6   keywords     25545 non-null  object 
 7   power        25545 non-null  int64  
 8   toughness    25545 non-null  int64  
 9   rarity       25545 non-null  object 
 10  reserved     25545 non-null  bool   
 11  released_at  25545 non-null  int64  
 12  booster      25545 non-null  bool   
 13  usd          25489 non-null  float64
dtypes: bool(3), float64(1), int64(4), object(6)
memory usage: 2.2+ MB


In [17]:
df.head(3)

,cmc,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,rarity,reserved,released_at,booster,usd
0,2,[W],False,[Artifact],[Equipment],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,[Equip],0,0,common,False,10096,True,0.02
1,4,[U],False,[Creature],"[Bird, Rogue]",Flying\nWhen Aarakocra Sneak enters the battle...,[Flying],1,4,common,False,10418,True,0.06
2,5,"[B, R, U]",True,[Creature],"[Astartes, Warrior]",Trample\nMark of Chaos Ascendant — During your...,"[Mark of Chaos Ascendant, Trample]",5,5,mythic,False,10537,False,2.81


In [18]:
def process_oracle_text(oracle_text):
    color_mapping = {
        'R': 'Red',
        'U': 'Blue',
        'G': 'Green',
        'B': 'Black',
        'W': 'White',
        'C': 'Colorless',
    }

    pattern = re.compile(r'\{([^\}]+)\}')

    def replace_color(match):
        code = match.group(1)
        if code == 'T':
            return 'Tap'
        elif code == 'X':
            return 'X'
        else:
            colors = code.split('/')
            color_text = ' or '.join(color_mapping.get(c, f'{c}') for c in colors).strip()
            return color_text
    processed_text = pattern.sub(replace_color, oracle_text)

    processed_text = re.sub(r'\.', '. ', processed_text)
    processed_text = re.sub(r'\{(\d+)}', lambda x: f'{x.group(1)} ', processed_text)
    processed_text = re.sub("[\(\[].*?[\)\]]", "", processed_text)
    processed_text = processed_text.replace('\n', ' ')

    return processed_text

oracle_text_example = "{R} {U} {G} {B} {W} {C} {R/G} {T} {X} This card has {2} abilities."
processed_text = process_oracle_text(oracle_text_example)
print(processed_text)


Red Blue Green Black White Colorless Red or Green Tap X This card has 2 abilities. 


In [19]:
df['oracle_text'] = df['oracle_text'].apply(process_oracle_text)

In [20]:
df.head(3)

,cmc,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,rarity,reserved,released_at,booster,usd
0,2,[W],False,[Artifact],[Equipment],Equipped creature gets +2/+2. Equip 3,[Equip],0,0,common,False,10096,True,0.02
1,4,[U],False,[Creature],"[Bird, Rogue]",Flying When Aarakocra Sneak enters the battlef...,[Flying],1,4,common,False,10418,True,0.06
2,5,"[B, R, U]",True,[Creature],"[Astartes, Warrior]",Trample Mark of Chaos Ascendant — During your ...,"[Mark of Chaos Ascendant, Trample]",5,5,mythic,False,10537,False,2.81


In [21]:
df.drop(df[df['oracle_text'].isna()].index, inplace=True)
df = df.reset_index(drop=True)

In [26]:
df.isna().value_counts()

cmc    colors  legendary  types  sub_types  oracle_text  keywords  power  toughness  rarity  reserved  released_at  booster  usd  
False  False   False      False  False      False        False     False  False      False   False     False        False    False    25489
                                                                                                                             True        56
Name: count, dtype: int64

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25545 entries, 0 to 25544
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   cmc          25545 non-null  int64  
 1   colors       25545 non-null  object 
 2   legendary    25545 non-null  bool   
 3   types        25545 non-null  object 
 4   sub_types    25545 non-null  object 
 5   oracle_text  25545 non-null  object 
 6   keywords     25545 non-null  object 
 7   power        25545 non-null  int64  
 8   toughness    25545 non-null  int64  
 9   rarity       25545 non-null  object 
 10  reserved     25545 non-null  bool   
 11  released_at  25545 non-null  int64  
 12  booster      25545 non-null  bool   
 13  usd          25489 non-null  float64
dtypes: bool(3), float64(1), int64(4), object(6)
memory usage: 2.2+ MB


In [23]:
directory_path = '/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/data'
df.to_csv(os.path.join(directory_path, 'clean_cards.csv'), index=False)